## Теоретическая часть


1. Ответьте на вопросы:  
В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?  
Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы  




2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:  
1) Какой датасет используют авторы?  
2) Что используют в качестве признаков?  
3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко  

## Практическая часть


In [1]:
%matplotlib inline

import lightfm
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import datetime

from src import metrics

/Users/macbook/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
# import data
# utils functions like in webinar

### Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

### Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

In [3]:
PATH_TO_DATA = '../урок 2/retail_train.csv'
PATH_TO_USERS_FEATURES_DATA = '../урок 2/hh_demographic.csv'
PATH_TO_ITEMS_FEATURES_DATA = '../урок 2/product.csv'

data = pd.read_csv(PATH_TO_DATA)

users_features = pd.read_csv(PATH_TO_USERS_FEATURES_DATA)
items_features = pd.read_csv(PATH_TO_ITEMS_FEATURES_DATA)

users_features.columns = [column.lower() for column in users_features.columns]
items_features.columns = [column.lower() for column in items_features.columns]

users_features.rename(columns={'household_key': 'user_id'}, inplace=True)
items_features.rename(columns={'product_id': 'item_id'}, inplace=True)

test_size_weeks = 3

weeks_ratio = data['week_no'] - test_size_weeks

data_train = data[data['week_no'] < weeks_ratio]
data_test = data[data['week_no'] >= weeks_ratio]

In [4]:
users_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [5]:
items_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
items_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               92353 non-null  int64 
 1   manufacturer          92353 non-null  int64 
 2   department            92353 non-null  object
 3   brand                 92353 non-null  object
 4   commodity_desc        92353 non-null  object
 5   sub_commodity_desc    92353 non-null  object
 6   curr_size_of_product  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


In [7]:
users_features.household_size_desc = \
    users_features.household_size_desc.apply(lambda x: int(x) if x.isdigit() else int(x.split('+')[0]))

In [8]:
users_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age_desc             801 non-null    object
 1   marital_status_code  801 non-null    object
 2   income_desc          801 non-null    object
 3   homeowner_desc       801 non-null    object
 4   hh_comp_desc         801 non-null    object
 5   household_size_desc  801 non-null    int64 
 6   kid_category_desc    801 non-null    object
 7   user_id              801 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 50.2+ KB


In [9]:
def print_uniq_feat(df):
    for column in df.select_dtypes(include='object').columns.values:

        uniques = df[column].unique()

        print('\n')
        print(f'Название столбца: {column}\n')
        print(uniques)
        print('\n' + '*' * 50)

In [10]:
print_user_uniq_feat = print_uniq_feat(users_features)



Название столбца: age_desc

['65+' '45-54' '25-34' '35-44' '19-24' '55-64']

**************************************************


Название столбца: marital_status_code

['A' 'U' 'B']

**************************************************


Название столбца: income_desc

['35-49K' '50-74K' '25-34K' '75-99K' 'Under 15K' '100-124K' '15-24K'
 '125-149K' '150-174K' '250K+' '175-199K' '200-249K']

**************************************************


Название столбца: homeowner_desc

['Homeowner' 'Unknown' 'Renter' 'Probable Renter' 'Probable Owner']

**************************************************


Название столбца: hh_comp_desc

['2 Adults No Kids' '2 Adults Kids' 'Single Female' 'Unknown'
 'Single Male' '1 Adult Kids']

**************************************************


Название столбца: kid_category_desc

['None/Unknown' '1' '2' '3+']

**************************************************


In [11]:
print_items_uniq_feat = print_uniq_feat(items_features)



Название столбца: department

['GROCERY' 'MISC. TRANS.' 'PASTRY' 'DRUG GM' 'MEAT-PCKGD' 'SEAFOOD-PCKGD'
 'PRODUCE' 'NUTRITION' 'DELI' 'COSMETICS' 'MEAT' 'FLORAL'
 'TRAVEL & LEISUR' 'SEAFOOD' 'MISC SALES TRAN' 'SALAD BAR' 'KIOSK-GAS'
 'ELECT &PLUMBING' 'GRO BAKERY' 'GM MERCH EXP' 'FROZEN GROCERY'
 'COUP/STR & MFG' 'SPIRITS' 'GARDEN CENTER' 'TOYS' 'CHARITABLE CONT'
 'RESTAURANT' 'RX' 'PROD-WHS SALES' 'MEAT-WHSE' 'DAIRY DELI' 'CHEF SHOPPE'
 'HBC' 'DELI/SNACK BAR' 'PORK' 'AUTOMOTIVE' 'VIDEO RENTAL' ' '
 'CNTRL/STORE SUP' 'HOUSEWARES' 'POSTAL CENTER' 'PHOTO' 'VIDEO'
 'PHARMACY SUPPLY']

**************************************************


Название столбца: brand

['National' 'Private']

**************************************************


Название столбца: commodity_desc

['FRZN ICE' 'NO COMMODITY DESCRIPTION' 'BREAD' 'FRUIT - SHELF STABLE'
 'COOKIES/CONES' 'SPICES & EXTRACTS' 'VITAMINS' 'BREAKFAST SWEETS'
 'PNT BTR/JELLY/JAMS' 'ICE CREAM/MILK/SHERBTS' 'MAGAZINE' 'AIR CARE'
 'CHEESE' 'SHO

In [12]:
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [16]:
data['sales_value_after_retail_disc'] = data.sales_value + data.retail_disc
sales_per_basket = data.groupby('basket_id')['sales_value_after_retail_disc'].sum().reset_index()

In [17]:
sales_per_basket

,basket_id,sales_value_after_retail_disc
0,26984851472,4.62
1,26984851516,9.17
2,26984896261,9.19
3,26984905972,0.47
4,26984945254,3.53
...,...,...
256996,41655046833,27.37
256997,41655180249,26.12
256998,41655820646,36.84
256999,41655829421,3.38


In [24]:
data_uniques = data.groupby('basket_id')['']()

AttributeError: 'DataFrameGroupBy' object has no attribute 'unique'

In [21]:
data_uniques

,user_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_after_retail_disc
basket_id,,,,,,,,,,,,
26984851472,True,True,True,True,True,True,False,True,True,False,False,True
26984851516,True,True,True,True,True,True,False,True,True,False,False,True
26984896261,True,True,True,True,True,True,False,True,True,False,False,True
26984905972,True,True,True,True,True,True,True,True,True,False,False,True
26984945254,True,True,True,True,True,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
41655046833,True,True,True,True,True,True,False,True,True,False,False,True
41655180249,True,True,True,True,True,True,False,True,True,False,False,True
41655820646,True,True,True,True,True,True,False,True,True,False,False,True


In [14]:
# data = data.merge(items_features, how='left', on='item_id')
# data = data.merge(users_features, how='left', on='user_id')
# data

### LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 17%  
У Вас, скорее всего, возникнет проблема со временем обучения. Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

*задание необязательно
